In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
from sklearn.preprocessing import LabelEncoder

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

C:\Users\OWNER\AppData\Local\Temp\ipykernel_21692\2213573283.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is alread

True

In [2]:
file_path = r'C:\Users\OWNER\Desktop\AnyoneAI\final_project\assignment\alpha2_dataset.csv'

df = pd.read_csv(file_path)
df = df.fillna(pd.NA)

print(df.head())
df.info()

                                              name      type  price  \
0                Duracell - AAA Batteries (4-Pack)  HardGood   5.49   
1  Duracell - AA 1.5V CopperTop Batteries (4-Pack)  HardGood   5.49   
2                 Duracell - AA Batteries (8-Pack)  HardGood   7.49   
3            Energizer - MAX Batteries AA (4-Pack)  HardGood   4.99   
4                  Duracell - C Batteries (4-Pack)  HardGood   8.99   

                                         description manufacturer  \
0  Compatible with select electronic devices; AAA...     Duracell   
1  Long-lasting energy; DURALOCK Power Preserve t...     Duracell   
2  Compatible with select electronic devices; AA ...     Duracell   
3  4-pack AA alkaline batteries; battery tester i...    Energizer   
4  Compatible with select electronic devices; C s...     Duracell   

                                           url              parent_category  \
0                duracell aaa batteries 4 pack  Connected Home & Housewares   


In [3]:
# stemmer, lemmatizer and stopwords
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from typing import Optional

# Initialize NLTK resources
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return 'a'  # Adjective
    elif tag.startswith('V'):
        return 'v'  # Verb
    elif tag.startswith('N'):
        return 'n'  # Noun
    elif tag.startswith('R'):
        return 'r'  # Adverb
    else:
        return 'n'  # Default to noun if not recognized

def remove_extra_new_lines(text):

    if pd.isnull(text):  # check if text is nan
        return ''  # replace with an empty string

    clean_text = [i for i in str(text).splitlines() if i.strip()]
    clean_text = ' '.join(clean_text)
    return clean_text

def remove_extra_whitespace(text: str) -> str:

    spaceless_text = re.sub(r'\s+', ' ', text)
    return spaceless_text

def remove_special_chars(text: str, remove_digits: Optional[bool] = False) -> str:

    if remove_digits:
        pattern = r'[^a-zA-Z\s]'
    else:
        pattern = r'[^a-zA-Z0-9\s]'

    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

def normalize_text(text):

    text = remove_extra_new_lines(text)

    text = remove_extra_whitespace(text)

    text = remove_special_chars(text, remove_digits=False)

    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalpha() and token.lower() not in stop_words]
    tagged_tokens = pos_tag(tokens)
    lemmas = [lemmatizer.lemmatize(token, get_wordnet_pos(tag)) for token, tag in tagged_tokens]

    return ' '.join(lemmas)

In [4]:
normalization = ['name', 'description']
for column in normalization:
    df[column + '_normalized'] = df[column].apply(normalize_text)

print(df.shape)

(50041, 13)


In [5]:
#df['sub_category_1'].fillna('0', inplace=True)
#df['sub_category_2'].fillna('0', inplace=True)
#df['sub_category_3'].fillna('0', inplace=True)
#df['sub_category_4'].fillna('0', inplace=True)


In [6]:
print(df.head())
df.shape
X = df.drop(columns=['parent_category'])
y = df['parent_category']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

                                              name      type  price  \
0                Duracell - AAA Batteries (4-Pack)  HardGood   5.49   
1  Duracell - AA 1.5V CopperTop Batteries (4-Pack)  HardGood   5.49   
2                 Duracell - AA Batteries (8-Pack)  HardGood   7.49   
3            Energizer - MAX Batteries AA (4-Pack)  HardGood   4.99   
4                  Duracell - C Batteries (4-Pack)  HardGood   8.99   

                                         description manufacturer  \
0  Compatible with select electronic devices; AAA...     Duracell   
1  Long-lasting energy; DURALOCK Power Preserve t...     Duracell   
2  Compatible with select electronic devices; AA ...     Duracell   
3  4-pack AA alkaline batteries; battery tester i...    Energizer   
4  Compatible with select electronic devices; C s...     Duracell   

                                           url              parent_category  \
0                duracell aaa batteries 4 pack  Connected Home & Housewares   


In [7]:
df.shape

X_1 = df.drop(columns=['sub_category_1'])
y_1 = df['sub_category_1']
y_1.fillna('missing', inplace=True)

label_encoder = LabelEncoder()
y_1_encoded = label_encoder.fit_transform(y_1)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1_encoded, test_size=0.2, random_state=42)

In [8]:
df.shape
X_2 = df.drop(columns=['sub_category_2'])
y_2 = df['sub_category_2']
y_2.fillna('missing', inplace=True)
label_encoder = LabelEncoder()
y_2_encoded = label_encoder.fit_transform(y_2)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2_encoded, test_size=0.2, random_state=42)

In [9]:
df.shape
X_3 = df.drop(columns=['sub_category_3'])
y_3 = df['sub_category_3']
y_3.fillna('missing', inplace=True)

label_encoder = LabelEncoder()
y_3_encoded = label_encoder.fit_transform(y_3)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3, y_3_encoded, test_size=0.2, random_state=42)

In [10]:
df.shape
X_4 = df.drop(columns=['sub_category_4'])
y_4 = df['sub_category_4']
y_4.fillna('missing', inplace=True)

label_encoder = LabelEncoder()
y_4_encoded = label_encoder.fit_transform(y_4)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X_4, y_4_encoded, test_size=0.2, random_state=42)

In [11]:
print(df.head())
df.shape

                                              name      type  price  \
0                Duracell - AAA Batteries (4-Pack)  HardGood   5.49   
1  Duracell - AA 1.5V CopperTop Batteries (4-Pack)  HardGood   5.49   
2                 Duracell - AA Batteries (8-Pack)  HardGood   7.49   
3            Energizer - MAX Batteries AA (4-Pack)  HardGood   4.99   
4                  Duracell - C Batteries (4-Pack)  HardGood   8.99   

                                         description manufacturer  \
0  Compatible with select electronic devices; AAA...     Duracell   
1  Long-lasting energy; DURALOCK Power Preserve t...     Duracell   
2  Compatible with select electronic devices; AA ...     Duracell   
3  4-pack AA alkaline batteries; battery tester i...    Energizer   
4  Compatible with select electronic devices; C s...     Duracell   

                                           url              parent_category  \
0                duracell aaa batteries 4 pack  Connected Home & Housewares   


(50041, 13)

In [12]:
print(X_train.shape)
print( X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(X_train['name_normalized'])

(40032, 12)
(10009, 12)
(40032,)
(10009,)
9233                            star fox preowned nintendo
25631    pioneer networkready ultra hd passthrough av h...
19030                     evolve ultimate edition xbox one
12044    joby pro series ultraplate quickrelease plate ...
18967    aluratek bump w home audio speaker system ipod...
                               ...                        
11284    samsung class diag lead curved smart ultra hd ...
44732    hifonics brutus class mono mosfet subwoofer am...
38158    mobile edge premium laptop backpack apple macb...
860                                 presonus presonus gray
15795      insignia portable bluetooth stereo speaker blue
Name: name_normalized, Length: 40032, dtype: object


In [13]:
y_train

array([55,  5, 55, ..., 16, 43,  5])

One Hot Encoder y Scaler


Stage 1

In [14]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Define columns
categorical_columns = ['type', 'manufacturer']
numerical_columns = ['price']
text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train[categorical_columns] = X_train[categorical_columns].fillna('missing')
X_test[categorical_columns] = X_test[categorical_columns].fillna('missing')

# One-hot encode categorical features
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train[categorical_columns])
X_test_encoded = encoder.transform(X_test[categorical_columns])

# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded:", X_train_encoded.shape)
print("Data type of X_train_encoded:", type(X_train_encoded))
print("Data type of elements in X_train_encoded:", X_train_encoded.dtype)
print("Shape of X_test_encoded:", X_test_encoded.shape)
print("Data type of X_test_encoded:", type(X_test_encoded))
print("Data type of elements in X_test_encoded:", X_test_encoded.dtype)

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[numerical_columns])
X_test_scaled = scaler.transform(X_test[numerical_columns])

# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled:", X_train_scaled.shape)
print("Data type of elements in X_train_scaled:", X_train_scaled.dtype)
print("Shape of X_test_scaled:", X_test_scaled.shape)
print("Data type of elements in X_test_scaled:", X_test_scaled.dtype)

Shape of X_train_encoded: (40032, 2195)
Data type of X_train_encoded: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded: float64
Shape of X_test_encoded: (10009, 2195)
Data type of X_test_encoded: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded: float64
Shape of X_train_scaled: (40032, 1)
Data type of elements in X_train_scaled: float64
Shape of X_test_scaled: (10009, 1)
Data type of elements in X_test_scaled: float64


Stage 2

In [15]:
# Define columns
categorical_columns_1 = ['type', 'manufacturer', 'parent_category']
# numerical_columns = ['price']
# text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train_1[categorical_columns_1] = X_train_1[categorical_columns_1].fillna('missing')
X_test_1[categorical_columns_1] = X_test_1[categorical_columns_1].fillna('missing')

# One-hot encode categorical features
encoder_1 = OneHotEncoder(handle_unknown='ignore')
X_train_encoded_1 = encoder_1.fit_transform(X_train_1[categorical_columns_1])
X_test_encoded_1 = encoder_1.transform(X_test_1[categorical_columns_1])

# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded_1:", X_train_encoded_1.shape)
print("Data type of X_train_encoded_1:", type(X_train_encoded_1))
print("Data type of elements in X_train_encoded_1:", X_train_encoded_1.dtype)
print("Shape of X_test_encoded_1:", X_test_encoded_1.shape)
print("Data type of X_test_encoded_1:", type(X_test_encoded_1))
print("Data type of elements in X_test_encoded_1:", X_test_encoded_1.dtype)

# Scale numerical features
scaler_1 = StandardScaler()
X_train_scaled_1 = scaler_1.fit_transform(X_train_1[numerical_columns])
X_test_scaled_1 = scaler_1.transform(X_test_1[numerical_columns])

# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled_1:", X_train_scaled_1.shape)
print("Data type of elements in X_train_scaled_1:", X_train_scaled_1.dtype)
print("Shape of X_test_scaled_1:", X_test_scaled_1.shape)
print("Data type of elements in X_test_scaled_1:", X_test_scaled_1.dtype)

Shape of X_train_encoded_1: (40032, 2253)
Data type of X_train_encoded_1: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded_1: float64
Shape of X_test_encoded_1: (10009, 2253)
Data type of X_test_encoded_1: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded_1: float64
Shape of X_train_scaled_1: (40032, 1)
Data type of elements in X_train_scaled_1: float64
Shape of X_test_scaled_1: (10009, 1)
Data type of elements in X_test_scaled_1: float64


Stage 3

In [16]:
# Define columns
categorical_columns_2 = ['type', 'manufacturer', 'parent_category', 'sub_category_1']
# numerical_columns = ['price']
# text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train_2[categorical_columns_2] = X_train_2[categorical_columns_2].fillna('missing')
X_test_2[categorical_columns_2] = X_test_2[categorical_columns_2].fillna('missing')

# One-hot encode categorical features
encoder_2 = OneHotEncoder(handle_unknown='ignore')
X_train_encoded_2 = encoder_2.fit_transform(X_train_2[categorical_columns_2])
X_test_encoded_2 = encoder_2.transform(X_test_2[categorical_columns_2])

# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded_2:", X_train_encoded_2.shape)
print("Data type of X_train_encoded_2:", type(X_train_encoded_2))
print("Data type of elements in X_train_encoded_2:", X_train_encoded_2.dtype)
print("Shape of X_test_encoded_2:", X_test_encoded_2.shape)
print("Data type of X_test_encoded_2:", type(X_test_encoded_2))
print("Data type of elements in X_test_encoded_2:", X_test_encoded_2.dtype)

# Scale numerical features
scaler_2 = StandardScaler()
X_train_scaled_2 = scaler_2.fit_transform(X_train_2[numerical_columns])
X_test_scaled_2 = scaler_2.transform(X_test_2[numerical_columns])

# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled_2:", X_train_scaled_2.shape)
print("Data type of elements in X_train_scaled_2:", X_train_scaled_2.dtype)
print("Shape of X_test_scaled_2:", X_test_scaled_2.shape)
print("Data type of elements in X_test_scaled_2:", X_test_scaled_2.dtype)

Shape of X_train_encoded_2: (40032, 2398)
Data type of X_train_encoded_2: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded_2: float64
Shape of X_test_encoded_2: (10009, 2398)
Data type of X_test_encoded_2: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded_2: float64
Shape of X_train_scaled_2: (40032, 1)
Data type of elements in X_train_scaled_2: float64
Shape of X_test_scaled_2: (10009, 1)
Data type of elements in X_test_scaled_2: float64


Stage 4

In [17]:
# Define columns
categorical_columns_3 = ['type', 'manufacturer', 'parent_category', 'sub_category_1', 'sub_category_2']
# numerical_columns = ['price']
# text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train_3[categorical_columns_3] = X_train_3[categorical_columns_3].fillna('missing')
X_test_3[categorical_columns_3] = X_test_3[categorical_columns_3].fillna('missing')

# One-hot encode categorical features
encoder_3 = OneHotEncoder(handle_unknown='ignore')
X_train_encoded_3 = encoder_3.fit_transform(X_train_3[categorical_columns_3])
X_test_encoded_3 = encoder_3.transform(X_test_3[categorical_columns_3])

# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded_3:", X_train_encoded_3.shape)
print("Data type of X_train_encoded_3:", type(X_train_encoded_3))
print("Data type of elements in X_train_encoded_3:", X_train_encoded_3.dtype)
print("Shape of X_test_encoded_3:", X_test_encoded_3.shape)
print("Data type of X_test_encoded_3:", type(X_test_encoded_3))
print("Data type of elements in X_test_encoded_3:", X_test_encoded_3.dtype)

# Scale numerical features
scaler_3 = StandardScaler()
X_train_scaled_3 = scaler_3.fit_transform(X_train_3[numerical_columns])
X_test_scaled_3 = scaler_3.transform(X_test_3[numerical_columns])

# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled_3:", X_train_scaled_3.shape)
print("Data type of elements in X_train_scaled_3:", X_train_scaled_3.dtype)
print("Shape of X_test_scaled_3:", X_test_scaled_3.shape)
print("Data type of elements in X_test_scaled_3:", X_test_scaled_3.dtype)

Shape of X_train_encoded_3: (40032, 2943)
Data type of X_train_encoded_3: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded_3: float64
Shape of X_test_encoded_3: (10009, 2943)
Data type of X_test_encoded_3: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded_3: float64
Shape of X_train_scaled_3: (40032, 1)
Data type of elements in X_train_scaled_3: float64
Shape of X_test_scaled_3: (10009, 1)
Data type of elements in X_test_scaled_3: float64


Stage 5

In [18]:
# Define columns
categorical_columns_4 = ['type', 'manufacturer', 'parent_category', 'sub_category_1', 'sub_category_2', 'sub_category_3']
# numerical_columns = ['price']
# text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train_4[categorical_columns_4] = X_train_4[categorical_columns_4].fillna('missing')
X_test_4[categorical_columns_4] = X_test_4[categorical_columns_4].fillna('missing')

# One-hot encode categorical features
encoder_4 = OneHotEncoder(handle_unknown='ignore')
X_train_encoded_4 = encoder_4.fit_transform(X_train_4[categorical_columns_4])
X_test_encoded_4 = encoder_4.transform(X_test_4[categorical_columns_4])

# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded_4:", X_train_encoded_4.shape)
print("Data type of X_train_encoded_4:", type(X_train_encoded_4))
print("Data type of elements in X_train_encoded_4:", X_train_encoded_4.dtype)
print("Shape of X_test_encoded_4:", X_test_encoded_4.shape)
print("Data type of X_test_encoded_4:", type(X_test_encoded_4))
print("Data type of elements in X_test_encoded_4:", X_test_encoded_4.dtype)

# Scale numerical features
scaler_4 = StandardScaler()
X_train_scaled_4 = scaler_4.fit_transform(X_train_4[numerical_columns])
X_test_scaled_4 = scaler_4.transform(X_test_4[numerical_columns])

# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled_4:", X_train_scaled_4.shape)
print("Data type of elements in X_train_scaled_4:", X_train_scaled_4.dtype)
print("Shape of X_test_scaled_4:", X_test_scaled_4.shape)
print("Data type of elements in X_test_scaled_4:", X_test_scaled_4.dtype)

Shape of X_train_encoded_4: (40032, 3632)
Data type of X_train_encoded_4: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded_4: float64
Shape of X_test_encoded_4: (10009, 3632)
Data type of X_test_encoded_4: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded_4: float64
Shape of X_train_scaled_4: (40032, 1)
Data type of elements in X_train_scaled_4: float64
Shape of X_test_scaled_4: (10009, 1)
Data type of elements in X_test_scaled_4: float64


In [19]:
from scipy.sparse import hstack

# Stage 1
X_train_processed = hstack([X_train_encoded, X_train_scaled]).astype(np.float32).toarray()
X_test_processed = hstack([X_test_encoded, X_test_scaled]).astype(np.float32).toarray()

# Stage 2
X_train_processed_1 = hstack([X_train_encoded_1, X_train_scaled_1]).astype(np.float32).toarray()
X_test_processed_1 = hstack([X_test_encoded_1, X_test_scaled_1]).astype(np.float32).toarray()

# Stage 3
X_train_processed_2 = hstack([X_train_encoded_2, X_train_scaled_2]).astype(np.float32).toarray()
X_test_processed_2 = hstack([X_test_encoded_2, X_test_scaled_2]).astype(np.float32).toarray()

# Stage 4
X_train_processed_3 = hstack([X_train_encoded_3, X_train_scaled_3]).astype(np.float32).toarray()
X_test_processed_3 = hstack([X_test_encoded_3, X_test_scaled_3]).astype(np.float32).toarray()

# Stage 5
X_train_processed_4 = hstack([X_train_encoded_4, X_train_scaled_4]).astype(np.float32).toarray()
X_test_processed_4 = hstack([X_test_encoded_4, X_test_scaled_4]).astype(np.float32).toarray()



In [20]:
# Dim 1
print("Data type of X_train_processed:", X_train_processed.dtype)
print("Data type of X_test_processed:", X_test_processed.dtype)
print("X_train_processed shape:", X_train_processed.shape)
print("X_test_processed shape:", X_test_processed.shape)

Data type of X_train_processed: float32
Data type of X_test_processed: float32
X_train_processed shape: (40032, 2196)
X_test_processed shape: (10009, 2196)


In [21]:
# Dim 2
print("Data type of X_train_processed_1:", X_train_processed_1.dtype)
print("Data type of X_test_processed_1:", X_test_processed_1.dtype)
print("X_train_processed_1 shape:", X_train_processed_1.shape)
print("X_test_processed_1 shape:", X_test_processed_1.shape)

Data type of X_train_processed_1: float32
Data type of X_test_processed_1: float32
X_train_processed_1 shape: (40032, 2254)
X_test_processed_1 shape: (10009, 2254)


In [22]:
# Dim 3
print("Data type of X_train_processed_2:", X_train_processed_2.dtype)
print("Data type of X_test_processed_2:", X_test_processed_2.dtype)
print("X_train_processed_2 shape:", X_train_processed_2.shape)
print("X_test_processed_2 shape:", X_test_processed_2.shape)

Data type of X_train_processed_2: float32
Data type of X_test_processed_2: float32
X_train_processed_2 shape: (40032, 2399)
X_test_processed_2 shape: (10009, 2399)


In [23]:
# Dim 4
print("Data type of X_train_processed_3:", X_train_processed_3.dtype)
print("Data type of X_test_processed_3:",X_test_processed_3.dtype)
print("X_train_processed_3 shape:", X_train_processed_3.shape)
print("X_test_processed_3 shape:", X_test_processed_3.shape)

Data type of X_train_processed_3: float32
Data type of X_test_processed_3: float32
X_train_processed_3 shape: (40032, 2944)
X_test_processed_3 shape: (10009, 2944)


In [24]:
# Dim 5
print("Data type of X_train_processed_4:", X_train_processed_4.dtype)
print("Data type of X_test_processed_4:", X_test_processed_4.dtype)
print("X_train_processed_4 shape:", X_train_processed_4.shape)
print("X_test_processed_4 shape:", X_test_processed_4.shape)

Data type of X_train_processed_4: float32
Data type of X_test_processed_4: float32
X_train_processed_4 shape: (40032, 3633)
X_test_processed_4 shape: (10009, 3633)


In [25]:
import numpy as np

# Define file paths to save the arrays
file_paths = {
    'X_train_processed.npy': X_train_processed,
    'X_test_processed.npy': X_test_processed,
    'X_train_processed_1.npy': X_train_processed_1,
    'X_test_processed_1.npy': X_test_processed_1,
    'X_train_processed_2.npy': X_train_processed_2,
    'X_test_processed_2.npy': X_test_processed_2,
    'X_train_processed_3.npy': X_train_processed_3,
    'X_test_processed_3.npy': X_test_processed_3,
    'X_train_processed_4.npy': X_train_processed_4,
    'X_test_processed_4.npy': X_test_processed_4
}

# Save each array
for file_name, array in file_paths.items():
    np.save(file_name, array)


In [26]:
X_train_name_embeddings_loaded = np.load('X_train_name_embeddings.npy', allow_pickle= True)
X_train_description_embeddings_loaded = np.load('X_train_description_embeddings.npy',allow_pickle= True)
X_test_name_embeddings_loaded = np.load('X_test_name_embeddings.npy', allow_pickle= True)
X_test_description_embeddings_loaded = np.load('X_test_description_embeddings.npy', allow_pickle= True)

# Load the saved NumPy arrays
X_train_processed_loaded = np.load('X_train_processed.npy', allow_pickle=True)
X_test_processed_loaded = np.load('X_test_processed.npy', allow_pickle=True)
X_train_processed_1_loaded = np.load('X_train_processed_1.npy', allow_pickle=True)
X_test_processed_1_loaded = np.load('X_test_processed_1.npy', allow_pickle=True)
X_train_processed_2_loaded = np.load('X_train_processed_2.npy', allow_pickle=True)
X_test_processed_2_loaded = np.load('X_test_processed_2.npy', allow_pickle=True)
X_train_processed_3_loaded = np.load('X_train_processed_3.npy', allow_pickle=True)
X_test_processed_3_loaded = np.load('X_test_processed_3.npy', allow_pickle=True)
X_train_processed_4_loaded = np.load('X_train_processed_4.npy', allow_pickle=True)
X_test_processed_4_loaded = np.load('X_test_processed_4.npy', allow_pickle=True)



In [27]:
X_train_name_last_hidden = np.array([x[0][-1] for x in X_train_name_embeddings_loaded])
X_train_description_last_hidden = np.array([x[0][-1] for x in X_train_description_embeddings_loaded])
X_test_name_last_hidden = np.array([x[0][-1] for x in X_test_name_embeddings_loaded])
X_test_description_last_hidden = np.array([x[0][-1] for x in X_test_description_embeddings_loaded])

In [28]:
print("First element of X_train_name_last_hidden:", X_train_name_last_hidden[0])

First element of X_train_name_last_hidden: [ 6.20537519e-01 -1.57028392e-01 -4.39105332e-01  5.87245941e-01
 -4.92567480e-01 -6.75922573e-01  3.66373003e-01 -7.91122556e-01
  6.63730741e-01  4.76211309e-02  4.97472771e-02 -3.77824754e-01
 -5.29451743e-02 -3.52970883e-03 -7.33473897e-01 -2.74465412e-01
 -8.19092095e-02 -1.52533606e-01  7.26244077e-02 -4.86062840e-02
  3.07591200e-01 -1.32735014e-01  8.47112298e-01  2.39423364e-01
  1.34860486e-01  3.51377934e-01 -4.78440404e-01 -1.80202276e-02
 -2.26639926e-01 -3.52599382e-01 -6.21600211e-01 -2.67526209e-01
 -1.92636084e-02  3.99675339e-01  1.55551210e-01 -8.61436129e-02
  3.39647025e-01 -4.60110493e-02 -5.71235001e-01 -4.14561182e-01
 -3.34018916e-01 -1.61173679e-02 -1.93653673e-01  6.32311583e-01
  5.47741950e-02 -6.00057304e-01  6.22461319e-01  3.42181236e-01
 -2.25496083e-01  4.89310294e-01  2.93806463e-01  1.60992384e-01
 -7.31535181e-02  8.67539346e-02  5.76151237e-02  1.21880889e-01
  2.94119507e-01 -4.34998453e-01  1.54902250e-0

In [29]:
print("Shape of X_train_name_last_hidden:", X_train_name_last_hidden.shape)
print("Data type of X_train_name_last_hidden:", X_train_name_last_hidden.dtype)

print("Shape of X_train_description_last_hidden:", X_train_description_last_hidden.shape)
print("Data type of X_train_description_last_hidden:", X_train_description_last_hidden.dtype)

print("Shape of X_test_name_last_hidden:", X_test_name_last_hidden.shape)
print("Data type of X_test_name_last_hidden:", X_test_name_last_hidden.dtype)

print("Shape of X_test_description_last_hidden:", X_test_description_last_hidden.shape)
print("Data type of X_test_description_last_hidden:", X_test_description_last_hidden.dtype)

Shape of X_train_name_last_hidden: (40032, 768)
Data type of X_train_name_last_hidden: float32
Shape of X_train_description_last_hidden: (40032, 768)
Data type of X_train_description_last_hidden: float32
Shape of X_test_name_last_hidden: (10009, 768)
Data type of X_test_name_last_hidden: float32
Shape of X_test_description_last_hidden: (10009, 768)
Data type of X_test_description_last_hidden: float32


In [30]:
X_train_concatenated = np.concatenate((X_train_name_last_hidden, X_train_description_last_hidden), axis=1)
X_test_concatenated = np.concatenate((X_test_name_last_hidden, X_test_description_last_hidden), axis=1)

In [31]:
print("Shape of X_train_concatenated:", X_train_concatenated.shape)
print("Data type of X_train_concatenated:", X_train_concatenated.dtype)

print("Shape of X_test_concatenated:", X_test_concatenated.shape)
print("Data type of X_test_concatenated:", X_test_concatenated.dtype)

Shape of X_train_concatenated: (40032, 1536)
Data type of X_train_concatenated: float32
Shape of X_test_concatenated: (10009, 1536)
Data type of X_test_concatenated: float32


In [32]:
# Stage 1
X_train_combined = np.concatenate((X_train_processed_loaded, X_train_concatenated), axis=1)
X_test_combined = np.concatenate((X_test_processed_loaded, X_test_concatenated), axis=1)

# Stage 2
X_train_combined_1 = np.concatenate((X_train_processed_1_loaded, X_train_concatenated), axis=1)
X_test_combined_1 = np.concatenate((X_test_processed_1_loaded, X_test_concatenated), axis=1)

# Stage 3
X_train_combined_2 = np.concatenate((X_train_processed_2_loaded, X_train_concatenated), axis=1)
X_test_combined_2 = np.concatenate((X_test_processed_2_loaded, X_test_concatenated), axis=1)

# Stage 4
X_train_combined_3 = np.concatenate((X_train_processed_3_loaded, X_train_concatenated), axis=1)
X_test_combined_3 = np.concatenate((X_test_processed_3_loaded, X_test_concatenated), axis=1)

# Stage 5
X_train_combined_4 = np.concatenate((X_train_processed_4_loaded, X_train_concatenated), axis=1)
X_test_combined_4 = np.concatenate((X_test_processed_4_loaded, X_test_concatenated), axis=1)


In [33]:
#Define num_clases

num_classes = len(df['parent_category'].unique())
num_classes_1 = len(df['sub_category_1'].unique())
num_classes_2 = len(df['sub_category_2'].unique())
num_classes_3 = len(df['sub_category_3'].unique())
num_classes_4 = len(df['sub_category_4'].unique())


In [34]:
print(num_classes_1)

149


In [44]:
from tensorflow.keras import callbacks
from tensorflow.keras.models import save_model
from sklearn.metrics import classification_report

# Define learning rate function
def lr_scheduler(epoch, lr):
    if epoch < 4:
        return lr * 0.94
    elif epoch < 8:
        return lr * 0.9
    elif epoch < 16: 
        return lr * 0.80
    elif epoch < 19:
        return lr * 0.70
    else:
        return lr * 0.6

# Define common callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler_callback = callbacks.LearningRateScheduler(lr_scheduler)

# Compile the model
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_combined.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Model fit
run_1 = model.fit(X_train_combined, y_train, epochs=60, batch_size=32,
                  validation_data=(X_test_combined, y_test),
                  callbacks=[early_stopping, lr_scheduler_callback])

save_model(model, 'model_1_preberttune.h5')

y_pred_probabilities = model.predict(X_test_combined)
y_pred = np.argmax(y_pred_probabilities, axis=1)

loss, accuracy = model.evaluate(X_test_combined, y_test)
print("Accuracy:", accuracy)

report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)


Epoch 1/60
1251/1251 [==============================] - 5s 3ms/step - loss: 1.0572 - accuracy: 0.7400 - val_loss: 0.4429 - val_accuracy: 0.8809 - lr: 9.4000e-04
Epoch 2/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.5946 - accuracy: 0.8409 - val_loss: 0.4867 - val_accuracy: 0.8600 - lr: 8.8360e-04
Epoch 3/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.4880 - accuracy: 0.8724 - val_loss: 0.3336 - val_accuracy: 0.9112 - lr: 8.3058e-04
Epoch 4/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.4272 - accuracy: 0.8873 - val_loss: 0.2568 - val_accuracy: 0.9350 - lr: 7.8075e-04
Epoch 5/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.3738 - accuracy: 0.9022 - val_loss: 0.2424 - val_accuracy: 0.9412 - lr: 7.0267e-04
Epoch 6/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.3428 - accuracy: 0.9109 - val_loss: 0.2655 - val_accuracy: 0.9308 - lr: 6.3241e-04
Epoch 7/60
1251/1251 [============

C:\Users\OWNER\AppData\Local\Temp\ipykernel_21692\2787510625.py:42: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model, 'model_1_preberttune.h5')


313/313 [==============================] - 0s 1ms/step - loss: 0.1893 - accuracy: 0.9564
Accuracy: 0.956439197063446
Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           4       0.98      0.98      0.98      1712
           5       0.93      0.96      0.94       715
           6       0.97      0.98      0.97        91
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1
           9       1.00      1.00      1.00         2
          11       0.95      0.97      0.96       649
          12       0.96      0.98      0.97       505
          13       0.00      0.00      0.00         3
          14       0.00      0.00      0.00         1
          15       0.96      0.97      0.97      1392
          16       0.94      0.95      0.95      1247
          17       0.93      0.93      0.93       890
          18       0.00      0.00      0.00      

C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [45]:
# Define learning rate function
def lr_scheduler(epoch, lr):
    if epoch < 4:
        return lr * 0.94
    elif epoch < 8:
        return lr * 0.9
    elif epoch < 16: 
        return lr * 0.80
    elif epoch < 19:
        return lr * 0.70
    else:
        return lr * 0.6

# Define common callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler_callback = callbacks.LearningRateScheduler(lr_scheduler)

# Create model
model_2 = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_combined_1.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes_1, activation='softmax')
])

# Compile model
model_2.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Fit model
history_2 = model_2.fit(X_train_combined_1, y_train_1, epochs=60, batch_size=32,
                        validation_data=(X_test_combined_1, y_test_1),
                        callbacks=[early_stopping, lr_scheduler_callback])

# Save model
save_model(model_2, 'model_2_preberttune.h5')

# Evaluate model
loss_2, accuracy_2 = model_2.evaluate(X_test_combined_1, y_test_1)
print("Accuracy for Stage 2 model:", accuracy_2)

# Generate classification report
y_pred_probabilities_2 = model_2.predict(X_test_combined_1)
y_pred_2 = np.argmax(y_pred_probabilities_2, axis=1)
report_2 = classification_report(y_test_1, y_pred_2)
print("Classification Report for Stage 2 model:")
print(report_2)



Epoch 1/60
1251/1251 [==============================] - 5s 4ms/step - loss: 1.8049 - accuracy: 0.6070 - val_loss: 0.5957 - val_accuracy: 0.8372 - lr: 9.4000e-04
Epoch 2/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.8570 - accuracy: 0.7700 - val_loss: 0.4661 - val_accuracy: 0.8711 - lr: 8.8360e-04
Epoch 3/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.6992 - accuracy: 0.8019 - val_loss: 0.4215 - val_accuracy: 0.8750 - lr: 8.3058e-04
Epoch 4/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.6005 - accuracy: 0.8276 - val_loss: 0.3212 - val_accuracy: 0.9034 - lr: 7.8075e-04
Epoch 5/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.5537 - accuracy: 0.8380 - val_loss: 0.3051 - val_accuracy: 0.9079 - lr: 7.0267e-04
Epoch 6/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.4990 - accuracy: 0.8512 - val_loss: 0.2627 - val_accuracy: 0.9162 - lr: 6.3241e-04
Epoch 7/60
1251/1251 [============

C:\Users\OWNER\AppData\Local\Temp\ipykernel_21692\1687609008.py:40: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model_2, 'model_2_preberttune.h5')


313/313 [==============================] - 0s 1ms/step - loss: 0.1920 - accuracy: 0.9452
Accuracy for Stage 2 model: 0.9452492594718933
313/313 [==============================] - 0s 1ms/step
Classification Report for Stage 2 model:
              precision    recall  f1-score   support

           1       0.92      0.94      0.93        35
           2       0.76      0.70      0.73        23
           3       1.00      1.00      1.00         6
           4       0.95      1.00      0.97        54
           5       0.91      0.92      0.92       128
           6       0.93      1.00      0.96        13
           7       1.00      1.00      1.00         8
           8       0.00      0.00      0.00         2
           9       1.00      0.99      0.99        79
          10       1.00      1.00      1.00         9
          11       0.96      0.94      0.95       156
          12       1.00      1.00      1.00         1
          13       0.83      0.55      0.66        71
          1

C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [46]:
# Stage 3

def lr_scheduler(epoch, lr):
    if epoch < 4:
        return lr * 0.94
    elif epoch < 8:
        return lr * 0.9
    elif epoch < 16: 
        return lr * 0.85
    elif epoch < 19:
        return lr * 0.75
    else:
        return lr * 0.7

# Define common callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler_callback = callbacks.LearningRateScheduler(lr_scheduler)


model_3 = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_combined_2.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes_2, activation='softmax')
])

# Compile model
model_3.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Fit model
history_3 = model_3.fit(X_train_combined_2, y_train_2, epochs=60, batch_size=32,
                        validation_data=(X_test_combined_2, y_test_2),
                        callbacks=[early_stopping, lr_scheduler_callback])

# Save model
save_model(model_3, 'model_3_preberttune.h5')

# Evaluate model
loss_3, accuracy_3 = model_3.evaluate(X_test_combined_2, y_test_2)
print("Accuracy for Stage 3 model:", accuracy_3)

# Generate classification report
y_pred_probabilities_3 = model_3.predict(X_test_combined_2)
y_pred_3 = np.argmax(y_pred_probabilities_3, axis=1)
report_3 = classification_report(y_test_2, y_pred_3)
print("Classification Report for Stage 3 model:")
print(report_3)



Epoch 1/60
1251/1251 [==============================] - 6s 4ms/step - loss: 2.9663 - accuracy: 0.4649 - val_loss: 1.7470 - val_accuracy: 0.6254 - lr: 9.4000e-04
Epoch 2/60
1251/1251 [==============================] - 5s 4ms/step - loss: 1.5692 - accuracy: 0.6442 - val_loss: 0.9047 - val_accuracy: 0.7719 - lr: 8.8360e-04
Epoch 3/60
1251/1251 [==============================] - 5s 4ms/step - loss: 1.2371 - accuracy: 0.6979 - val_loss: 0.6932 - val_accuracy: 0.8210 - lr: 8.3058e-04
Epoch 4/60
1251/1251 [==============================] - 4s 3ms/step - loss: 1.0676 - accuracy: 0.7298 - val_loss: 0.5935 - val_accuracy: 0.8368 - lr: 7.8075e-04
Epoch 5/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.9674 - accuracy: 0.7487 - val_loss: 0.5430 - val_accuracy: 0.8551 - lr: 7.0267e-04
Epoch 6/60
1251/1251 [==============================] - 5s 4ms/step - loss: 0.8983 - accuracy: 0.7654 - val_loss: 0.5427 - val_accuracy: 0.8568 - lr: 6.3241e-04
Epoch 7/60
1251/1251 [============

C:\Users\OWNER\AppData\Local\Temp\ipykernel_21692\3792186953.py:41: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model_3, 'model_3_preberttune.h5')


313/313 [==============================] - 1s 2ms/step - loss: 0.3778 - accuracy: 0.9008
Accuracy for Stage 3 model: 0.9007892608642578
313/313 [==============================] - 0s 1ms/step
Classification Report for Stage 3 model:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00        18
           4       0.00      0.00      0.00         1
           5       0.78      1.00      0.88        70
           6       1.00      0.33      0.50         3
           8       0.67      0.86      0.75         7
          10       0.50      1.00      0.67         6
          12       0.75      0.75      0.75         8
          13       1.00      1.00      1.00        10
          14       0.83      0.97      0.89        39
          15       1.00      0.83      0.91         6
          16       0.50      1.00      0.67         3
          1

C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric

In [47]:
# Stage 4
print("Running Stage 4 model...")

def lr_scheduler(epoch, lr):
    if epoch < 4:
        return lr * 0.94
    elif epoch < 8:
        return lr * 0.9
    elif epoch < 16: 
        return lr * 0.80
    elif epoch < 19:
        return lr * 0.70
    else:
        return lr * 0.6

# Define common callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler_callback = callbacks.LearningRateScheduler(lr_scheduler)

# Create model
model_4 = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_combined_3.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes_3, activation='softmax')
])

# Compile model
model_4.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Fit model
history_4 = model_4.fit(X_train_combined_3, y_train_3, epochs=60, batch_size=32,
                        validation_data=(X_test_combined_3, y_test_3),
                        callbacks=[early_stopping, lr_scheduler_callback])

# Save model
save_model(model_4, 'model_4_preberttune.h5')

# Evaluate model
loss_4, accuracy_4 = model_4.evaluate(X_test_combined_3, y_test_3)
print("Accuracy for Stage 4 model:", accuracy_4)

# Generate classification report
y_pred_probabilities_4 = model_4.predict(X_test_combined_3)
y_pred_4 = np.argmax(y_pred_probabilities_4, axis=1)
report_4 = classification_report(y_test_3, y_pred_4)
print("Classification Report for Stage 4 model:")
print(report_4)


Running Stage 4 model...
Epoch 1/30
1251/1251 [==============================] - 6s 4ms/step - loss: 2.7567 - accuracy: 0.5621 - val_loss: 1.3178 - val_accuracy: 0.7329 - lr: 9.4000e-04
Epoch 2/30
1251/1251 [==============================] - 5s 4ms/step - loss: 1.4781 - accuracy: 0.6983 - val_loss: 0.8923 - val_accuracy: 0.8070 - lr: 8.8360e-04
Epoch 3/30
1251/1251 [==============================] - 5s 4ms/step - loss: 1.1642 - accuracy: 0.7439 - val_loss: 0.6919 - val_accuracy: 0.8489 - lr: 8.3058e-04
Epoch 4/30
1251/1251 [==============================] - 5s 4ms/step - loss: 0.9945 - accuracy: 0.7685 - val_loss: 0.6251 - val_accuracy: 0.8496 - lr: 7.8075e-04
Epoch 5/30
1251/1251 [==============================] - 5s 4ms/step - loss: 0.8986 - accuracy: 0.7845 - val_loss: 0.5608 - val_accuracy: 0.8665 - lr: 7.0267e-04
Epoch 6/30
1251/1251 [==============================] - 5s 4ms/step - loss: 0.8236 - accuracy: 0.7990 - val_loss: 0.5089 - val_accuracy: 0.8765 - lr: 6.3241e-04
Epoch 7/3

C:\Users\OWNER\AppData\Local\Temp\ipykernel_21692\2139566038.py:42: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model_4, 'model_4_preberttune.h5')


313/313 [==============================] - 1s 2ms/step - loss: 0.3912 - accuracy: 0.9118
Accuracy for Stage 4 model: 0.9117794036865234
313/313 [==============================] - 1s 1ms/step
Classification Report for Stage 4 model:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           6       1.00      1.00      1.00        56
           8       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         3
          16       0.00      0.00      0.00         3
          17       0.31      1.00      0.47         8
          19       0.67      0.33      0.44         6
          20       0.00      0.00      0.00         2
          2

C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric

In [48]:
# Stage 5
print("Running Stage 5 model...")
def lr_scheduler(epoch, lr):
    if epoch < 4:
        return lr * 0.94
    elif epoch < 8:
        return lr * 0.9
    elif epoch < 16: 
        return lr * 0.80
    elif epoch < 19:
        return lr * 0.70
    else:
        return lr * 0.6

# Define common callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler_callback = callbacks.LearningRateScheduler(lr_scheduler)
# Create model
model_5 = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_combined_4.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes_4, activation='softmax')
])

# Compile model
model_5.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Fit model
history_5 = model_5.fit(X_train_combined_4, y_train_4, epochs=60, batch_size=32,
                        validation_data=(X_test_combined_4, y_test_4),
                        callbacks=[early_stopping, lr_scheduler_callback])

# Save model
save_model(model_5, 'model_5_preberttune.h5')

# Evaluate model
loss_5, accuracy_5 = model_5.evaluate(X_test_combined_4, y_test_4)
print("Accuracy for Stage 5 model:", accuracy_5)

# Generate classification report
y_pred_probabilities_5 = model_5.predict(X_test_combined_4)
y_pred_5 = np.argmax(y_pred_probabilities_5, axis=1)
report_5 = classification_report(y_test_4, y_pred_5)
print("Classification Report for Stage 5 model:")
print(report_5)


Running Stage 5 model...
Epoch 1/30
1251/1251 [==============================] - 7s 5ms/step - loss: 1.7161 - accuracy: 0.7560 - val_loss: 0.7289 - val_accuracy: 0.8572 - lr: 9.4000e-04
Epoch 2/30
1251/1251 [==============================] - 5s 4ms/step - loss: 0.7660 - accuracy: 0.8524 - val_loss: 0.5045 - val_accuracy: 0.8915 - lr: 8.8360e-04
Epoch 3/30
1251/1251 [==============================] - 5s 4ms/step - loss: 0.6383 - accuracy: 0.8648 - val_loss: 0.3853 - val_accuracy: 0.9058 - lr: 8.3058e-04
Epoch 4/30
1251/1251 [==============================] - 5s 4ms/step - loss: 0.5418 - accuracy: 0.8772 - val_loss: 0.3362 - val_accuracy: 0.9108 - lr: 7.8075e-04
Epoch 5/30
1251/1251 [==============================] - 5s 4ms/step - loss: 0.4812 - accuracy: 0.8842 - val_loss: 0.3017 - val_accuracy: 0.9208 - lr: 7.0267e-04
Epoch 6/30
1251/1251 [==============================] - 5s 4ms/step - loss: 0.4463 - accuracy: 0.8886 - val_loss: 0.2824 - val_accuracy: 0.9253 - lr: 6.3241e-04
Epoch 7/3

C:\Users\OWNER\AppData\Local\Temp\ipykernel_21692\3460058163.py:40: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model_5, 'model_5_preberttune.h5')


313/313 [==============================] - 0s 1ms/step - loss: 0.2115 - accuracy: 0.9409
Accuracy for Stage 5 model: 0.9408532381057739
313/313 [==============================] - 0s 1ms/step
Classification Report for Stage 5 model:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           2       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         4
           6       0.81      0.94      0.87        18
           8       0.00      0.00      0.00         2
           9       1.00      0.25      0.40         4
          12       1.00      1.00      1.00         5
          13       0.78      1.00      0.88        71
          14       1.00      0.80      0.89         5
          15       0.95      1.00      0.97        18
          16       0.00      0.00      0.00         4
          17       0.00      0.00      0.00         1
          1

C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f